In [129]:
transformation='JW'

In [130]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit import Aer

In [1355]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
## HAMILTONIAN start
Molecule = 'H2'
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-3g'

# Molecule = 'LiH'
# geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., 1.44))]
# basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian_PySCF(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation=transformation)
### HAMILTONIAN end

#####################################

print(QubitHamiltonian)


fci_energy = Hamilt.molecule.fci_energy
print(fci_energy)

(-4.083814987252351+0j) [] +
(-0.003929930732041322+0j) [X0 X1 Y2 Y3] +
(-0.0029440463277662417+0j) [X0 X1 Y2 Z3 Z4 Y5] +
(0.0018547889719190407+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
(-0.0029440463277662417+0j) [X0 X1 X3 X4] +
(0.001854788971919041+0j) [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
(-0.005350601465592326+0j) [X0 X1 Y4 Y5] +
(0.00011409127923189481+0j) [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
(0.0001140912792318948+0j) [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
(-0.002455120054590474+0j) [X0 X1 Y6 Y7] +
(-0.002455120054590474+0j) [X0 X1 Y8 Y9] +
(-0.0015818788065834752+0j) [X0 X1 Y10 Y11] +
(0.003929930732041322+0j) [X0 Y1 Y2 X3] +
(0.0029440463277662417+0j) [X0 Y1 Y2 Z3 Z4 X5] +
(-0.0018547889719190407+0j) [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
(-0.0029440463277662417+0j) [X0 Y1 Y3 X4] +
(0.001854788971919041+0j) [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
(0.005350601465592326+0j) [X0 Y1 Y4 X5] +
(-0.00011409127923189481+0j) [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
(0.0001140912792318948+0j) [X0 Y1 Y5

From Helgaker, T., P. Jorgensen,and J. Olsen (2014),Molecularelectronic-structure theory(John Wiley & Sons)
we known

$$H_{2}^{ground} = 0.9939| 1100\rangle - 0.1106| 0011\rangle$$

From group theory we know only double excitation terms are important!
We can see this from our ijab operators that qubits 1 and 3 are only acted on by $Z$ therefore experience no population changes... Can even see this from the answer if we write it in BK basis:

$$H_{2}^{BK} = 0.9939| 1000\rangle - 0.1106| 0010\rangle$$

See how qubits 1 and 3 always remain at 0!

therefore can **remove** these measurements from Hamiltonian... as we know what expectation values they should give!

In the case for this Hamiltonian only $I$ and $Z$ act on these qubits! So even easier to remove (expectation values = +1 (as act on $| 0\rangle$ state!), so just remove and add to relivent H terms!

LOOK at:

PHYS. REV. X, **8**, 031022 (2018)

In [1356]:
from quchem.Ansatz_Generator_Functions import *
n_electrons=Hamilt.molecule.n_electrons
n_qubits=Hamilt.molecule.n_qubits

ansatz_obj = Ansatz(n_electrons,n_qubits)
print('JW ground state = ', ansatz_obj.Get_JW_HF_state_in_OCC_basis())
print('BK ground state = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())

JW ground state =  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
BK ground state =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [1357]:
qubits_to_remove = Find_I_Z_indices_in_Hamiltonian(QubitHamiltonian, Hamilt.molecule.n_qubits)
print('qubits only acted on by I or Z:', qubits_to_remove)


input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
# Remove_Z_terms_from_Hamiltonian
NewQubitHamiltonian = Remove_Z_terms_from_Hamiltonian(
                                                    QubitHamiltonian, 
                                                    input_state,
                                                    qubits_to_remove,
                                                    check_reduction=True)
NewQubitHamiltonian

qubits only acted on by I or Z: []


(-4.083814987252351+0j) [] +
(-0.003929930732041322+0j) [X0 X1 Y2 Y3] +
(-0.0029440463277662417+0j) [X0 X1 Y2 Z3 Z4 Y5] +
(0.0018547889719190407+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
(-0.0029440463277662417+0j) [X0 X1 X3 X4] +
(0.001854788971919041+0j) [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
(-0.005350601465592326+0j) [X0 X1 Y4 Y5] +
(0.00011409127923189481+0j) [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
(0.0001140912792318948+0j) [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
(-0.002455120054590474+0j) [X0 X1 Y6 Y7] +
(-0.002455120054590474+0j) [X0 X1 Y8 Y9] +
(-0.0015818788065834752+0j) [X0 X1 Y10 Y11] +
(0.003929930732041322+0j) [X0 Y1 Y2 X3] +
(0.0029440463277662417+0j) [X0 Y1 Y2 Z3 Z4 X5] +
(-0.0018547889719190407+0j) [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
(-0.0029440463277662417+0j) [X0 Y1 Y3 X4] +
(0.001854788971919041+0j) [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
(0.005350601465592326+0j) [X0 Y1 Y4 X5] +
(-0.00011409127923189481+0j) [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
(0.0001140912792318948+0j) [X0 Y1 Y5

In [1358]:
qubitNo_re_label_dict, NewQubitHamiltonian_relabelled = Re_label_Hamiltonian(NewQubitHamiltonian)
NewQubitHamiltonian_relabelled

(-4.083814987252351+0j) [] +
(-0.003929930732041322+0j) [X0 X1 Y2 Y3] +
(-0.0029440463277662417+0j) [X0 X1 Y2 Z3 Z4 Y5] +
(0.0018547889719190407+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
(-0.0029440463277662417+0j) [X0 X1 X3 X4] +
(0.001854788971919041+0j) [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
(-0.005350601465592326+0j) [X0 X1 Y4 Y5] +
(0.00011409127923189481+0j) [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
(0.0001140912792318948+0j) [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
(-0.002455120054590474+0j) [X0 X1 Y6 Y7] +
(-0.002455120054590474+0j) [X0 X1 Y8 Y9] +
(-0.0015818788065834752+0j) [X0 X1 Y10 Y11] +
(0.003929930732041322+0j) [X0 Y1 Y2 X3] +
(0.0029440463277662417+0j) [X0 Y1 Y2 Z3 Z4 X5] +
(-0.0018547889719190407+0j) [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
(-0.0029440463277662417+0j) [X0 Y1 Y3 X4] +
(0.001854788971919041+0j) [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
(0.005350601465592326+0j) [X0 Y1 Y4 X5] +
(-0.00011409127923189481+0j) [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
(0.0001140912792318948+0j) [X0 Y1 Y5

# Find what new FCI energy is
- here should be the same as true answer!

In [1359]:
from openfermion import qubit_operator_sparse
from scipy.sparse import csc_matrix
new_Molecular_H_MATRIX =  csc_matrix(qubit_operator_sparse(NewQubitHamiltonian_relabelled))

from scipy.sparse.linalg import eigs
try:
    eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
except:
    from scipy.linalg import eig
    eig_values, eig_vectors = eig(new_Molecular_H_MATRIX.todense())
    
new_FCI_Energy = min(eig_values)


index = np.where(eig_values==new_FCI_Energy)[0][0]
ground_state_vector = eig_vectors[:, index]

print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', fci_energy)
print(np.isclose(new_FCI_Energy, fci_energy))

new_FCI =  (-7.88057428180283+1.8054014402081412e-16j) VS old FCI: -7.880574281802676
True


# HF + UCCSD

In [136]:
input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
new_input_state = np.delete(input_state, qubits_to_remove)

n_qubits_new = len(new_input_state)
new_input_state

array([1, 1, 0, 0])

In [137]:
ansatz_obj.Get_ia_and_ijab_terms()

print('ia standard')
print(ansatz_obj.Sec_Quant_CC_ia_Fermi_ops)

print('ijab standard')
print(ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

### single trotter step
ansatz_obj.UCCSD_single_trotter_step(transformation,
                                     List_FermiOps_ia=ansatz_obj.Sec_Quant_CC_ia_Fermi_ops,
                                     List_FermiOps_ijab=ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

print('')
print('ia standard')
print(ansatz_obj.Second_Quant_CC_single_Trot_list_ia)
print('ijab standard')
print(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab)

ia standard
[-1.0 [0^ 2] +
1.0 [2^ 0], -1.0 [1^ 3] +
1.0 [3^ 1]]
ijab standard
[-1.0 [0^ 1^ 2 3] +
1.0 [3^ 2^ 1 0]]

ia standard
[-0.5j [X0 Z1 Y2] +
0.5j [Y0 Z1 X2], -0.5j [X1 Z2 Y3] +
0.5j [Y1 Z2 X3]]
ijab standard
[0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]]


In [138]:
new_CC_ia_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ia,
                                                      qubits_to_remove)
print('ia reduced')
print(new_CC_ia_single_trot)

new_CC_ijab_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab,
                                                      qubits_to_remove)
print('ijab reduced')
print(new_CC_ijab_single_trot)


relabelled_new_CC_ia_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ia_single_trot)
relabelled_new_CC_ijab_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ijab_single_trot)

print('')
print('ia reduced relabelled')
print(relabelled_new_CC_ia_single_trot)
print('')
print('ijab reduced relabelled')
print(relabelled_new_CC_ijab_single_trot)

ia reduced
[-0.5j [X0 Z1 Y2] +
0.5j [Y0 Z1 X2], -0.5j [X1 Z2 Y3] +
0.5j [Y1 Z2 X3]]
ijab reduced
[0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]]

ia reduced relabelled
[-0.5j [X0 Z1 Y2] +
0.5j [Y0 Z1 X2], -0.5j [X1 Z2 Y3] +
0.5j [Y1 Z2 X3]]

ijab reduced relabelled
[0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]]


# Ansatz Circuit

In [11]:
# for op in relabelled_new_CC_ia_single_trot[1]:
#     print(op)
# print(list(op.terms.keys())[0])

In [36]:
# UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(new_input_state,
#                                      relabelled_new_CC_ia_single_trot, 
#                                      relabelled_new_CC_ijab_single_trot)

# theta_ia = [0 for _ in range(len(relabelled_new_CC_ia_single_trot))]
# theta_ijab = [0 for _ in range(len(relabelled_new_CC_ijab_single_trot))]

# UCCSD_ansatz_Q_Circ =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
#                                         Theta_param_list_ia=theta_ia, 
#                                          Theta_param_list_ijab=theta_ijab,
#                                          ia_first=True)
# UCCSD_ansatz_Q_Circ

# Graph

In [139]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(NewQubitHamiltonian_relabelled)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)

anti_commuting_sets


Building Graph Edges: 100%|##########| 15/15 [00:00<00:00, 804.30it/s]


{0: [(0.1659278503377034+0j) [Z0 Z3]],
 1: [(0.12062523483390414+0j) [Z0 Z2]],
 2: [(0.1744128761226159+0j) [Z2 Z3]],
 3: [(0.12062523483390414+0j) [Z1 Z3]],
 4: [(0.16868898170361207+0j) [Z0 Z1]],
 5: [(-0.09706626816762878+0j) []],
 6: [(0.1659278503377034+0j) [Z1 Z2]],
 7: [(0.045302615503799264+0j) [X0 Y1 Y2 X3], (0.17141282644776892+0j) [Z1]],
 8: [(0.045302615503799264+0j) [Y0 X1 X2 Y3], (0.1714128264477689+0j) [Z0]],
 9: [(-0.045302615503799264+0j) [Y0 Y1 X2 X3], (-0.22343153690813564+0j) [Z2]],
 10: [(-0.045302615503799264+0j) [X0 X1 Y2 Y3],
  (-0.22343153690813558+0j) [Z3]]}

In [14]:
# from scipy.sparse import kron as sparse_kron
# from scipy.sparse import csc_matrix

# class standard_VQE_lin_alg():
#     def __init__(self, Hamiltonian, qubit_CCSD_ia_list, qubit_CCSD_ijab_list, HF_input_state, n_qubits):
#         self.Hamiltonian_matrix=get_sparse_operator(Hamiltonian)
#         self.number_ia_terms=len(qubit_CCSD_ia_list)
#         self.number_ijab_terms=len(qubit_CCSD_ijab_list)
#         self.HF_input_state=HF_input_state
        
#         self.UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(HF_input_state,
#                                                      qubit_CCSD_ia_list, 
#                                                      qubit_CCSD_ijab_list)
        
#         zero_qubit_state = csc_matrix(np.array([[1],[0]]), dtype=complex)
#         self.all_zero_input= reduce(sparse_kron, [zero_qubit_state for _ in range(n_qubits)])

    
#     def calc_E(self, theta_ia_ijab_combined):
        
#         theta_ia=theta_ia_ijab_combined[:self.number_ia_terms]
#         theta_ijab=theta_ia_ijab_combined[self.number_ia_terms:]
        
#         ansatz_cirq_circuit =self.UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
#                                                                                 Theta_param_list_ia=theta_ia, 
#                                                                                  Theta_param_list_ijab=theta_ijab,
#                                                                                  ia_first=True)
        
#         ansatz_mat = csc_matrix(ansatz_cirq_circuit.unitary())
        
#         ansatz_state_ket=ansatz_mat.dot(self.all_zero_input)
#         ansatz_state_bra = ansatz_state_ket.transpose().conj()
        
#         Energy =  ansatz_state_bra.dot(self.Hamiltonian_matrix.dot(ansatz_state_ket)).todense().item(0)
#         return Energy.real

In [15]:
# VQE_lin_alg = standard_VQE_lin_alg(NewQubitHamiltonian_relabelled,
#                      relabelled_new_CC_ia_single_trot,
#                      relabelled_new_CC_ijab_single_trot,
#                      new_input_state,
#                      n_qubits_new)

# import random
# random_input = [random.uniform(0, 2*np.pi) for _ in range(len(relabelled_new_CC_ia_single_trot)+ len(relabelled_new_CC_ijab_single_trot))]

# # VQE_lin_alg.calc_E([0,0,0])
# VQE_lin_alg.calc_E(random_input)

In [16]:
# from quchem.Scipy_Optimizer import *

In [17]:
# random_input = [random.uniform(0, 2*np.pi) for _ in range(len(relabelled_new_CC_ia_single_trot)+ len(relabelled_new_CC_ijab_single_trot))]


# STANDARD_optimization = Optimizer(VQE_lin_alg.calc_E,
#                    random_input,
# #                    args=(relabelled_reduced_Qubit_Hamiltonian,anti_commuting_sets),
#                    args=(),
#                    method= 'Nelder-Mead',#Custom_Adam, 
#                    jac=None,
#                    hess=None,
#                    hessp=None,
#                       bounds=None,
#                    constraints=None,
#                    tol=1e-8, 
#                    display_convergence_message=True,
#                    display_steps=True)

# STANDARD_optimization.get_env(50)
# STANDARD_optimization.plot_convergence()
# STANDARD_optimization.optimized_result

In [140]:
# optimal_ansatz_H2_JW_params = STANDARD_optimization.optimized_result.x
optimal_ansatz_H2_JW_params=np.array([ 3.14243787e+00, -2.56378500e-05,  3.25441274e+00])

UCCSD_ansatz_Q_Circ =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
                                        Theta_param_list_ia=optimal_ansatz_H2_JW_params[:2], 
                                         Theta_param_list_ijab=optimal_ansatz_H2_JW_params[2:],
                                         ia_first=True)

zero_state = np.array([[1],[0]])
zero_input_state = reduce(np.kron, [zero_state for _ in range(4)])
ground_state_VQE = UCCSD_ansatz_Q_Circ.unitary().dot(zero_input_state)

NameError: name 'UCCSD_ansatz_Q_Circ_obj' is not defined

In [ ]:
np.around(ground_state_VQE, 3).reshape(16,1)

In [ ]:
np.around(ground_state_vector, 3).reshape(16,1)

In [105]:
def measure_state_vector_standard_VQE(n_shots, Pauli_operator, Ansatz_state, n_qubits):
    
    I_circuit = cirq.Circuit([cirq.I.on(cirq.LineQubit(i)) for i in range(n_qubits)])

    circ_obj = Change_PauliWord_measurement_to_Z_basis(Pauli_operator)
    circuit = cirq.Circuit(cirq.decompose_once(
        (circ_obj(*cirq.LineQubit.range(circ_obj.num_qubits())))))
    
    full_circuit=cirq.Circuit(I_circuit, circuit)
    final_state = full_circuit.unitary().dot(Ansatz_state)
    
    qubit_list, _ = zip(*(list(*Pauli_operator.terms.keys())))
    
    meas_outputs = cirq.sample_state_vector(final_state, indices=qubit_list, repetitions=n_shots)
    
    meas_dict={}  
    for binary_out in meas_outputs:
        bin_arr_str = np.char.mod('%d', binary_out)
        bin_str = "".join(bin_arr_str)
        if bin_str in meas_dict.keys():
            meas_dict[bin_str]+=1
        else:
            meas_dict[bin_str]=1
    
    return meas_dict

measure_state_vector_standard_VQE(100, list(NewQubitHamiltonian_relabelled)[-2], ground_state_vector, n_qubits_new)
# measure_state_vector_standard_VQE(10, QubitOperator('X0 X1 X2 X3 X4 X5 X6 X7 X8 X9 X10'), ground_state_vector, n_qubits_new)


{'01': 40, '10': 31, '00': 9, '11': 20}

In [106]:
def calc_exp_pauliword(count_dict):
    # takes correct part of bitstring when all lines measured

    n_zeros = 0
    n_ones = 0

    for bitstring in count_dict:
        
        parity_m_term = sum([int(bit) for bit in bitstring]) % 2

        if parity_m_term == 0:
            n_zeros += count_dict[bitstring]
        elif parity_m_term == 1:
            n_ones += count_dict[bitstring]
        else:
            raise ValueError('state {} not allowed'.format(measure_term))

    expectation_value = (n_zeros - n_ones) / (n_zeros + n_ones)

    return expectation_value

In [107]:
from tqdm.notebook import tqdm

N_SHOTS=10000

E_list=[]
for qubitOp in tqdm(list(NewQubitHamiltonian_relabelled)):
    for PauliWord, const in qubitOp.terms.items():
        if PauliWord:
            output_dict = measure_state_vector_standard_VQE(N_SHOTS, qubitOp, ground_state_vector, n_qubits_new)
            exp_val = calc_exp_pauliword(output_dict)

            E_list.append(exp_val*const)
        else:
            E_list.append(const)
sum(E_list)

(-1.1371346406895007+0j)

# convert everything to qiskit circuits!
- note indexing goes from right to left in qiskit!

In [141]:
from quchem_ibm.Qiskit_Chemistry import *

In [142]:
q_reg = QuantumRegister(len(new_input_state))
qcirc = QuantumCircuit(q_reg)
HF_state_IBM_circuit(new_input_state, q_reg, qcirc).draw()

q20_0: ─────
            
q20_1: ─────
       ┌───┐
q20_2: ┤ X ├
       ├───┤
q20_3: ┤ X ├
       └───┘

In [143]:
# exp PauliWord

testOp=QubitOperator('X0 Y1 Z2', 1j)
theta= np.pi/2

q_reg = QuantumRegister(3)
qcirc = QuantumCircuit(q_reg)

test_circuit = exp_PauliWord_circuit_IBM(testOp,theta, q_reg, qcirc, 3)

print(test_circuit.draw())
    

                                                                     
q21_0: ──────────────■───────────────────────────────■───────────────
       ┌──────────┐┌─┴─┐                           ┌─┴─┐┌───────────┐
q21_1: ┤ RX(pi/2) ├┤ X ├──■─────────────────────■──┤ X ├┤ RX(-pi/2) ├
       └──┬───┬───┘└───┘┌─┴─┐┌───────────────┐┌─┴─┐├───┤└───────────┘
q21_2: ───┤ H ├─────────┤ X ├┤ My_Rz(-3.142) ├┤ X ├┤ H ├─────────────
          └───┘         └───┘└───────────────┘└───┘└───┘             


In [111]:
test = full_exponentiated_PauliWord_circuit(testOp, theta)
cirq_circuit = cirq.Circuit(cirq.decompose_once((test(*cirq.LineQubit.range(test.num_qubits())))))
cirq_circuit

0: ───H──────────@────────────────────@───H───────────
                 │                    │
1: ───Rx(0.5π)───X───@────────────@───X───Rx(-0.5π)───
                     │            │
2: ──────────────────X───Rz(-π)───X───────────────────

In [26]:
test_circuit.draw()

q1_0: ──────────────■───────────────────────────────■───────────────
      ┌──────────┐┌─┴─┐                           ┌─┴─┐┌───────────┐
q1_1: ┤ RX(pi/2) ├┤ X ├──■─────────────────────■──┤ X ├┤ RX(-pi/2) ├
      └──┬───┬───┘└───┘┌─┴─┐┌───────────────┐┌─┴─┐├───┤└───────────┘
q1_2: ───┤ H ├─────────┤ X ├┤ My_Rz(-3.142) ├┤ X ├┤ H ├─────────────
         └───┘         └───┘└───────────────┘└───┘└───┘

In [112]:
from qiskit.quantum_info import Operator
exp_qiskit_mat = Operator(test_circuit)
cirq_matrix = cirq_circuit.unitary()

np.allclose(exp_qiskit_mat.data, cirq_matrix)

True

In [113]:
from scipy.sparse.linalg import expm
sparse_mat = get_sparse_operator(testOp)
mat = expm(sparse_mat*theta)

np.allclose(mat.todense(), exp_qiskit_mat.data)

True

In [114]:
# measure in Z basis
test_circuit_final = change_basis_for_Z_measure(testOp, q_reg, test_circuit, 3)
test_circuit_final.draw()

»
q15_0: ──────────────■───────────────────────────────■───────────────»
       ┌──────────┐┌─┴─┐                           ┌─┴─┐┌───────────┐»
q15_1: ┤ RX(pi/2) ├┤ X ├──■─────────────────────■──┤ X ├┤ RX(-pi/2) ├»
       └──┬───┬───┘└───┘┌─┴─┐┌───────────────┐┌─┴─┐├───┤└───┬───┬───┘»
q15_2: ───┤ H ├─────────┤ X ├┤ My_Rz(-3.142) ├┤ X ├┤ H ├────┤ H ├────»
          └───┘         └───┘└───────────────┘└───┘└───┘    └───┘    »
«                   
«q15_0: ────────────
«       ┌──────────┐
«q15_1: ┤ RX(pi/2) ├
«       └──────────┘
«q15_2: ────────────
«

In [699]:
N_qubits = len(new_input_state)
# arb state initialization
# https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/circuits/3_summary_of_quantum_operations.ipynb

q_reg = QuantumRegister(N_qubits)
qcirc = QuantumCircuit(q_reg)


qcirc=arb_state_initalize_circuit(ground_state_vector, q_reg, qcirc)
print(qcirc)


backend = Aer.get_backend('statevector_simulator')
job = execute(qcirc, backend)
qc_state = job.result().get_statevector(qcirc)
qc_state

         »
q3492_0: »
         »
q3492_1: »
         »
q3492_2: »
         »
q3492_3: »
         »
«         ┌───────────────────────────────────────────────────────────────────────────────┐
«q3492_0: ┤0                                                                              ├
«         │                                                                               │
«q3492_1: ┤1                                                                              ├
«         │  initialize(0,0,0,-0.10393+-0.043171j,0,0,0,0,0,0,0,0,0.91763+0.38116j,0,0,0) │
«q3492_2: ┤2                                                                              ├
«         │                                                                               │
«q3492_3: ┤3                                                                              ├
«         └───────────────────────────────────────────────────────────────────────────────┘


array([-1.28077301e-18-1.52652395e-16j,  3.23036983e-19+2.23945281e-17j,
        3.83816821e-17+5.60735746e-18j, -1.03934474e-01-4.31711890e-02j,
        3.03653044e-17+3.62205618e-17j,  4.56676675e-18-1.04428598e-16j,
        7.28080447e-17-5.81617375e-17j,  2.98712442e-17+8.43058614e-18j,
        1.04858039e-17-3.59578329e-17j, -4.71332633e-17-2.30182585e-17j,
       -3.90008609e-18+8.48323747e-18j, -4.94951938e-17-1.94805841e-18j,
        9.17634496e-01+3.81157192e-01j,  2.17477455e-18-2.69327189e-18j,
       -7.74660442e-17-4.14133717e-19j,  1.15905703e-16-7.17348740e-17j])

In [707]:
xx.data[0]

(<qiskit.circuit.reset.Reset at 0x7f4e8b5e7a50>,
 [Qubit(QuantumRegister(4, 'q99'), 1)],
 [])

In [703]:
type(xx.data[0][0])

qiskit.circuit.reset.Reset

In [733]:
# import qiskit

xx = qcirc.decompose()#.decompose().decompose().decompose().decompose().decompose().decompose().decompose()
# for index, tup in enumerate(xx.data):
#     op_type, _, _=tup
# #     print(op_type)
#     if isinstance(op_type, qiskit.circuit.reset.Reset):
#         xx.data.pop(index)
#         print('rem')
# xx.draw()
xx.data=xx.data[4:]
xx.draw()

Operator(xx).data

array([[-6.93889390e-18+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        -1.12543887e-01+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         9.93646755e-01+0.00000000e+00j, -2.77555756e-17+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j, -6.93889390e-18-3.85185989e-34j,
         0.00000000e+00+0.00000000e+00j, -1.12543887e-01+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         2.77555756e-17+1.54074396e-3

In [726]:
[1,2,3,4,5,6][3:]

[4, 5, 6]

In [650]:
ket_qi = qcirc.Statevector.from_instruction(qcirc)

# Operator(qcirc.decompose().decompose().decompose().decompose())

AttributeError: 'QuantumCircuit' object has no attribute 'Statevector'

In [116]:
def calc_exp_pauliword(count_dict, PauliWord):
    # takes correct part of bitstring when all lines measured

    qubitNos, PauliStrs = zip(*list(*PauliWord.terms.keys()))
    n_zeros = 0
    n_ones = 0

    for bitstring in count_dict:
        measure_term = np.take([int(bit) for bit in bitstring], qubitNos)

        parity_m_term = sum(measure_term) % 2

        if parity_m_term == 0:
            n_zeros += count_dict[bitstring]
        elif parity_m_term == 1:
            n_ones += count_dict[bitstring]
        else:
            raise ValueError('state {} not allowed'.format(measure_term))

    expectation_value = (n_zeros - n_ones) / (n_zeros + n_ones)

    return expectation_value

In [146]:
from tqdm.notebook import tqdm

In [118]:
len(ground_state_vector)

4

In [148]:
n_qubits = int(np.log2(len(ground_state_vector)))
backend=Aer.get_backend('qasm_simulator')
n_shots=1024

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)

perfect_ansatz_circ=arb_state_initalize_circuit(ground_state_vector, q_reg, qcirc)


E_list=[]
# for qubitOp in NewQubitHamiltonian_relabelled:
for qubitOp in tqdm(NewQubitHamiltonian_relabelled, ascii=True, desc='performing VQE'):
    for PauliWord, const in qubitOp.terms.items():
        if PauliWord:
            full_circuit = change_basis_for_Z_measure(qubitOp, q_reg,
                                                      perfect_ansatz_circ,
                                                      n_qubits)
            full_circuit.measure_all()
            
            job = execute(full_circuit, backend, shots=n_shots)
            result = job.result()
            exp_counts_dict = result.get_counts(full_circuit)
            
            exp_val = calc_exp_pauliword(exp_counts_dict, qubitOp)
            E_list.append(exp_val*const)
        else:
            E_list.append(const)
sum(E_list)

(-1.1386368918111232+0j)

In [149]:
def sparse_all_close(a,b, rtol=1e-10, atol=1e-8):
    c=np.abs(np.abs(a-b)-rtol*np.abs(b))
    return c.max()<=atol

from scipy.sparse import csr_matrix
test = csr_matrix([[1,2], [1,1]])
test2 = csr_matrix([[2,2], [0,1]])
sparse_all_close(test, test, atol=3)

True

(2, 1)

In [121]:
np.dot(test, test2)

<2x2 sparse matrix of type '<class 'numpy.longlong'>'
	with 4 stored elements in Compressed Sparse Row format>

In [150]:
from quchem.Unitary_partitioning_Seq_Rot import *
from scipy.sparse.linalg import expm
# from scipy.sparse import kron as sparse_kron


def Build_reduction_circuit_seq_rot_IBM(anti_commuting_set, S_index, N_qubits, check_reduction=False):
    """
    Function to build R_S (make up of all R_SK terms)

    Args:
        anti_commuting_set(list): list of anti commuting QubitOperators
        S_index(int): index for Ps in anti_commuting_set list
        check_reduction (optional, bool): use linear algebra to check that 𝑅s† 𝐻s 𝑅s == 𝑃s
    returns:
        full_RS_circuit(cirq.Circuit): Q_circuit for R_s operator
        Ps (QubitOperator): Pauli_S operator with cofactor of 1!
        gamma_l (float): normalization term

    """

    X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(anti_commuting_set, S_index)
    
    R_sl_mat_list=[]
    for X_sk_Op, theta_sk in X_sk_theta_sk_list:

        pauliword_X_sk = list(X_sk_Op.terms.keys())[0]
        const_X_sk = list(X_sk_Op.terms.values())[0]

        
        sparse_X_sk = get_sparse_operator(QubitOperator(pauliword_X_sk, -1j), n_qubits=N_qubits)
        R_sl_mat = expm(sparse_X_sk*(theta_sk / 2 * const_X_sk))
        R_sl_mat_list.append(R_sl_mat)

        
    R_S_matrix = reduce(np.dot, R_sl_mat_list[::-1]) # note need this in reverse order!

    if check_reduction:

        H_S=QubitOperator()
        for op in full_normalised_set['PauliWords']:
            H_S+=op
        H_S_matrix=get_sparse_operator(H_S, n_qubits=N_qubits)
        
        
        Ps_mat = get_sparse_operator(Ps, n_qubits=N_qubits)

        reduction_mat = R_S_matrix.dot(H_S_matrix.dot(R_S_matrix.conj().transpose()))
        
        if not (sparse_all_close(Ps_mat, reduction_mat)):
#         if not (np.allclose(Ps_mat.todense(), reduction_mat.todense())):
            print('reduction circuit incorrect...   𝑅s 𝐻s 𝑅s† != 𝑃s')

        
    return R_S_matrix, Ps, gamma_l


In [154]:
N_qubits = len(new_input_state)
S_index=0
set_key=9

R_s_mat, Ps, gamma_l=Build_reduction_circuit_seq_rot_IBM(anti_commuting_sets[set_key],
                                    S_index,
                                    N_qubits,
                                    check_reduction=True)

In [152]:
def IBM_gate_from_matrix(unitary_matrix, q_circuit, gate_str=None):
    
    if isinstance(unitary_matrix, scipy.sparse.csc.csc_matrix):
        unitary_matrix=unitary_matrix.toarray()
    
    
    q_circuit=q_circuit.copy()
    N_qubits = q_circuit.num_qubits
    
    if unitary_matrix.shape != (2**N_qubits, 2**N_qubits):
        raise ValueError('defined matrix is incorrect dimention')
    
    qubit_list = [i for i in range(N_qubits)]
    custom_gate = UnitaryGate(unitary_matrix, label='Custom Gate' if gate_str is None else '{}'.format(gate_str))
    
    del unitary_matrix
    q_circuit.append(custom_gate, qubit_list)
    return q_circuit

In [155]:
N_qubits = len(new_input_state)
q_reg = QuantumRegister(N_qubits)
qcirc = QuantumCircuit(q_reg)

qcirc = IBM_gate_from_matrix(R_s_mat, qcirc, gate_str='R_s')
qcirc.draw()

┌──────┐
q27_0: ┤0     ├
       │      │
q27_1: ┤1     ├
       │  R_s │
q27_2: ┤2     ├
       │      │
q27_3: ┤3     ├
       └──────┘

In [156]:
#Changing the simulator 
backend = Aer.get_backend('unitary_simulator')

#job execution and getting the result as an object
job = execute(qcirc, backend)
result = job.result()

#get the unitary matrix from the result object
unitary_form = result.get_unitary(qcirc, decimals=6)

np.allclose(unitary_form, np.around(R_s_mat.todense(), 6))

True

In [157]:
Seq_Rot_circuits=[]
N_qubits = len(new_input_state)
S_index=0

backend=Aer.get_backend('qasm_simulator')
n_shots=1024

q_reg = QuantumRegister(N_qubits)
qcirc = QuantumCircuit(q_reg)
perfect_ansatz_circ=arb_state_initalize_circuit(ground_state_vector, q_reg, qcirc)

E_list=[]
for set_key in tqdm(list(anti_commuting_sets.keys()), ascii=True, desc='performing VQE'):
    anti_set_list= anti_commuting_sets[set_key]
    if len(anti_set_list)>1:
        R_s_mat, Ps, gamma_l=Build_reduction_circuit_seq_rot_IBM(anti_set_list,
                                                                S_index,
                                                                N_qubits,
                                                                check_reduction=False)
        # unitary partitioning
        q_circuit = IBM_gate_from_matrix(R_s_mat, perfect_ansatz_circ, gate_str='R_s')
        
        full_circuit = change_basis_for_Z_measure(Ps, 
                                                  q_reg,
                                                  q_circuit,
                                                  N_qubits)
        full_circuit.measure_all()

        job = execute(full_circuit, backend, shots=n_shots)
        result = job.result()
        exp_counts_dict = result.get_counts(full_circuit)
        E_list.append(exp_val*gamma_l)
        
    else:
        qubitOp = anti_set_list[0]
        for PauliWord, const in qubitOp.terms.items():
            if PauliWord:
                full_circuit = change_basis_for_Z_measure(qubitOp, 
                                                          q_reg,
                                                          perfect_ansatz_circ,
                                                          N_qubits)
                full_circuit.measure_all()

                job = execute(full_circuit, backend, shots=n_shots)
                result = job.result()
                exp_counts_dict = result.get_counts(full_circuit)

                exp_val = calc_exp_pauliword(exp_counts_dict, qubitOp)
                E_list.append(exp_val*const)
            else:
                E_list.append(const)
print(sum(E_list))


(-1.137623207592054+0j)


# LCU

In [735]:
from qiskit.circuit.library.standard_gates import XGate, YGate, ZGate
N_QUBITS=2
q_reg = QuantumRegister(N_QUBITS+1)
qcirc = QuantumCircuit(q_reg)

X_gate = XGate().control(2)
X_gate.ctrl_state=0
qcirc.append(X_gate, [0,1,2])
qcirc.draw()

q5031_0: ──o──
           │  
q5031_1: ──o──
         ┌─┴─┐
q5031_2: ┤ X ├
         └───┘

In [179]:
X_gate.ctrl_state

3

In [158]:
from quchem.Unitary_partitioning_LCU_method import *

In [164]:
R_uncorrected, Pn, gamma_l = Get_R_op_list(anti_commuting_sets[9], 0)
R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)
R_corr_list

[1, 1j]

In [191]:
R_corrected_Op_list[0]

0.6706869158738031 []

In [241]:
from scipy.sparse import kron as sparse_kron
from scipy.sparse import csc_matrix

def control_Rl_gate_LCU(Pauli_Op_list, Cofactor_list, ancilla_state, n_qubits, ancilla_below_system=True):
    
    N_ancilla = int(np.ceil(np.log2(len(ancilla_state))))
    
    qubit_state = {'0': csc_matrix([[1],[0]]), '1': csc_matrix([[0],[1]])}
    
    
    R_l_op_list=[]
    control_index=0
    for index, qubitOp in enumerate(Pauli_Op_list):
        for PauliWord, const in qubitOp.terms.items(): 
            if PauliWord:
                control_string_binary = np.binary_repr(control_index, N_ancilla)
                control_state = reduce(sparse_kron, [qubit_state[bit]for bit in control_string_binary])
                control_mat = control_state.dot(control_state.conj().T)
                
                phase_correction=Cofactor_list[index]
                Pauli_matrix = get_sparse_operator(qubitOp, n_qubits=n_qubits)*phase_correction
                
                if ancilla_below_system:
                    final_mat = sparse_kron(Pauli_matrix, control_mat)
                else:
                    final_mat = sparse_kron(control_mat, Pauli_matrix)
                    
                R_l_op_list.append(final_mat)
                control_index+=1
            else:
                if isinstance(Cofactor_list[index], complex):
                    raise ValueError('Identity term is complex!')
    R_l_and_ancilla_matrix = reduce(np.dot, R_l_op_list[::-1]) # note need this in reverse order!
    total_N_qubits = n_qubits + N_ancilla
    return R_l_and_ancilla_matrix, N_ancilla
    


In [242]:
one, two = control_Rl_gate_LCU(R_corrected_Op_list, R_corr_list, ancilla_amplitudes, 4)
two

1

In [253]:
perfect_ansatz_circ.draw()

»
q45_0: »
       »
q45_1: »
       »
q45_2: »
       »
q45_3: »
       »
«       ┌───────────────────────────────────────────────────────────────────────────────┐
«q45_0: ┤0                                                                              ├
«       │                                                                               │
«q45_1: ┤1                                                                              ├
«       │  initialize(0,0,0,-0.10393+-0.043171j,0,0,0,0,0,0,0,0,0.91763+0.38116j,0,0,0) │
«q45_2: ┤2                                                                              ├
«       │                                                                               │
«q45_3: ┤3                                                                              ├
«       └───────────────────────────────────────────────────────────────────────────────┘

In [ ]:
# LCU_circuits=[]
N_qubits = len(new_input_state)
N_index=0

backend=Aer.get_backend('qasm_simulator')
n_shots=1024

q_reg = QuantumRegister(N_qubits)
qcirc = QuantumCircuit(q_reg)
perfect_ansatz_circ=arb_state_initalize_circuit(ground_state_vector, q_reg, qcirc)

E_list=[]
for set_key in tqdm(list(anti_commuting_sets.keys()), ascii=True, desc='performing VQE'):
    anti_set_list= anti_commuting_sets[set_key]
    if len(anti_set_list)>1:

        R_uncorrected, Pn, gamma_l = Get_R_op_list(anti_set_list, N_index)
        R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)
        # unitary partitioning
        
        LCU_mat, N_ancilla = control_Rl_gate_LCU(R_corrected_Op_list, R_corr_list, ancilla_amplitudes, N_qubits)
        
        q_reg_system_ancilla = QuantumRegister(N_qubits+N_ancilla)
        qcirc_system_ancilla = QuantumCircuit(q_reg_system_ancilla)
        ansatz_and_ancilla = arb_state_initalize_circuit(np.hstack((ground_state_vector, np.array(ancilla_amplitudes))),
                                                          q_reg_system_ancilla,
                                                          qcirc_system_ancilla)
        
        q_circuit = IBM_gate_from_matrix(ansatz_and_ancilla, perfect_ansatz_circ, gate_str='R_l_LCU')
        
        full_circuit = change_basis_for_Z_measure(Ps, 
                                                  q_reg,
                                                  q_circuit,
                                                  N_qubits)
        full_circuit.measure_all()

        job = execute(full_circuit, backend, shots=n_shots)
        result = job.result()
        exp_counts_dict = result.get_counts(full_circuit)
        E_list.append(exp_val*gamma_l)
        
    else:
        qubitOp = anti_set_list[0]
        for PauliWord, const in qubitOp.terms.items():
            if PauliWord:
                full_circuit = change_basis_for_Z_measure(qubitOp, 
                                                          q_reg,
                                                          perfect_ansatz_circ,
                                                          N_qubits)
                full_circuit.measure_all()

                job = execute(full_circuit, backend, shots=n_shots)
                result = job.result()
                exp_counts_dict = result.get_counts(full_circuit)

                exp_val = calc_exp_pauliword(exp_counts_dict, qubitOp)
                E_list.append(exp_val*const)
            else:
                E_list.append(const)
print(sum(E_list))

In [ ]:
def control_P_gate(PauliOp, cofactor, control_string_binary, n_qubits, n_ancilla):
    
    qubit_state = {'0': np.array([[1],[0]]), '1': np.array([[0],[1]])}
    
    ancilla_state = reduce(np.kron, [qubit_state[bit]for bit in control_string_binary])
    
    
    if Pstr == 'X':
        unitary_matrix = cofactor * np.array([[0, 1], [1, 0]], dtype=complex)
    elif Pstr == 'Y':
        unitary_matrix = cofactor * np.array([[0, -1j], [1j, 0]], dtype=complex)
    elif Pstr == 'Z':
        unitary_matrix = cofactor * np.array([[1, 0], [0, -1]], dtype=complex)
    else:
        raise ValueError('P_str is not a Pauli')

    return UnitaryGate(unitary_matrix, label='{}*{}'.format(cofactor, Pstr)).control(1)

In [263]:
A = np.eye(16)
A[:,0].shape

(16,)

In [287]:
A = np.eye(16)
A[:,0]=ground_state_vector
A[0,1]

/home/lex/anaconda3/envs/QuantumUpdated/lib/python3.7/site-packages/ipykernel_launcher.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  


0.0

In [288]:
def normalize(v):
    return v / np.sqrt(v.dot(v))

n = len(A)

# A[:, 0] = normalize(A[:, 0])

for i in range(1, n):
    Ai = A[:, i]
    for j in range(0, i):
        Aj = A[:, j]
        t = Ai.dot(Aj)
        Ai = Ai - t * Aj
#     A[:, i] = normalize(Ai)
    A[:, i] = Ai


In [289]:
sum(np.abs(i)**2 for i in ground_state_vector)

0.9999999999999996

In [348]:
ground_state_vector

array([-1.28077301e-18-1.52652395e-16j,  3.23036983e-19+2.23945281e-17j,
        3.83816821e-17+5.60735746e-18j, -1.03934474e-01-4.31711890e-02j,
        3.03653044e-17+3.62205618e-17j,  4.56676675e-18-1.04428598e-16j,
        7.28080447e-17-5.81617375e-17j,  2.98712442e-17+8.43058614e-18j,
        1.04858039e-17-3.59578329e-17j, -4.71332633e-17-2.30182585e-17j,
       -3.90008609e-18+8.48323747e-18j, -4.94951938e-17-1.94805841e-18j,
        9.17634496e-01+3.81157192e-01j,  2.17477455e-18-2.69327189e-18j,
       -7.74660442e-17-4.14133717e-19j,  1.15905703e-16-7.17348740e-17j])

In [294]:
np.trace(np.dot(A, A.conj().T))

14.862388609233339

In [383]:
xx = normalize(ground_state_vector)

np.allclose(ground_state_vector, xx)

True

In [379]:
def normalize(v):
    mag = sum(np.abs(v**2))
    return v / np.sqrt(mag)

def Get_Unitary_arb_state(arb_state):
    
    n_qubits = len(arb_state)
    
    A = np.eye(n_qubits, dtype=complex)
    A[:,0]=arb_state
    
    n = len(A)
    for i in range(1, n):
        Ai = A[:, i]
        for j in range(0, i):
            Aj = A[:, j]
            t = Ai.dot(Aj)
            Ai = Ai - t * Aj
        A[:, i] = normalize(Ai)
        
    return A
    

In [386]:
AA = Get_Unitary_arb_state(ground_state_vector)


np.allclose(ground_state_vector, AA[:,0])

True

In [396]:
A = np.eye(16, dtype=complex)
A[:,0]=ground_state_vector

q, r = np.linalg.qr(A, mode='complete')

In [397]:
np.allclose(q.dot(q.conj().T), np.eye(16, dtype=complex), atol=1e-1)

True

In [403]:
x = np.linalg.inv(r) * (q.T) * A
x

array([[-2.33027537e-32+1.95513068e-34j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        -0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
       [-0.00000000e+00+0.00000000e+00j,  1.00000000e+00+1.62169851e-50j,
        -0.00000000e+00+0.00000000e+00j,  0.00000000e+00-0.00000000e+00j,
        -0.00000000e+00+0.00000000e+00j, -0.00000000e+00+0.00000000e+00j,
        -0.00000000e+00+0.00000000e+00j,  0.00000000e+00-0.00000000e+00j,
         0.00000000e+00-0.00000000e+00j,  0.00000000e+00-0.00000000e+00j,
        -0.00000000e+00+0.00000000e+0

In [349]:
def normalize(v):
    return v / np.sqrt(v.dot(v))

n = len(A)

A[:, 0] = normalize(A[:, 0])

for i in range(1, n):
    Ai = A[:, i]
    for j in range(0, i):
        Aj = A[:, j]
        t = Ai.dot(Aj)
        Ai = Ai - t * Aj
    A[:, i] = normalize(Ai)


In [352]:
A[:, 1].dot(A[:, 2])

0j

In [347]:
np.sum(np.abs(unitary_check[:,4])**2)

1.0

In [338]:
np.allclose(unitary_check, np.eye(16, dtype=complex), atol=1e-1)

False

In [321]:
unitary_check[0,12]

(-0.022835626938481297-0.29946743562545575j)

In [320]:
np.where((np.around(unitary_check, 1) == np.eye(16)) == False)

(array([ 0,  0,  0,  3,  3,  3, 12, 12, 12, 12, 12, 12, 13, 13, 13, 14, 14,
        14, 14, 14, 14, 15, 15, 15, 15, 15, 15]),
 array([12, 14, 15, 12, 14, 15,  0,  3, 12, 13, 14, 15, 12, 14, 15,  0,  3,
        12, 13, 14, 15,  0,  3, 12, 13, 14, 15]))

In [409]:
x = np.array([1,0])
np.inner(x,x)

1

In [412]:
def normalize(v):
    mag = sum(np.abs(v**2))
    return v / np.sqrt(mag)

def Get_Unitary_arb_state(arb_state):
    
    n_qubits = len(arb_state)
    
    A = np.eye(n_qubits, dtype=complex)
    
    A[:,0]=arb_state
    
    n = len(A)
    for i in range(1,n):
        ortho_vec = A[:,i] - reduce(sum, [ np.inner(A[:,j], A[:,i])* A[:,j] for j in range(i)])
        ortho_vec=normalize(ortho_vec)
        print(np.dot(ortho_vec, A[:,i]))
        
Get_Unitary_arb_state(ground_state_vector)

(1-1.4468521591060976e-35j)
(1-1.7535178195327122e-17j)
(0.3213251961873559+0.17040489028224706j)
(1-1.01723442672571e-14j)
(1+3.417294801857174e-13j)
(1+1.488853867685371e-12j)
(1-1.7779686699390616e-11j)
(1+4.314233317593769e-10j)
(0.9999999999999987+9.304064283445443e-09j)
(0.9999999999999902-2.3985828392984584e-08j)
(0.9999999999317444+1.2585125567140364e-06j)
(-0.17642772173206478-0.17712498131078858j)
(0.9999999780756992+2.5623734594118328e-05j)
(0.9973110590986295+0.007314983085719318j)
(0.21699347199974453+0.04025097112520666j)


In [456]:
from scipy.stats import unitary_group

X = unitary_group.rvs(3)

A = np.eye(3, dtype=complex)
A[:,0]=normalize(X[:,0])

# n = len(A)
# for i in range(1,n):
    
#     ortho_vec = A[:,i] - reduce(sum, [ np.inner(A[:,j], A[:,i])* A[:,j] for j in range(i)])
#     ortho_vec=normalize(ortho_vec)
#     print(np.dot(ortho_vec, A[:,i]))

In [469]:
y2

array([-0.56000905-0.34242051j, -0.3791334 +0.11248444j,
       -0.61885471+0.17251673j])

In [470]:
v1 = X[:,0]
v2 = X[:,1]

y2 = v2 - np.inner(v2, v1)*v1
y2 = normalize(y2)
v1.dot(y2)

(-0.22812241882109283-0.10074317407744555j)

In [471]:
v1 = (1/np.sqrt(2)) * np.array([-1,1,0])
v2 = np.array([-1,0,1])

y2 = v2 - np.inner(v2, v1)*v1
y2 = normalize(y2)

v1.dot(y2)

1.1102230246251565e-16

In [437]:
v1

array([-0.13944433+0.17270927j, -0.43033076-0.08952676j,
       -0.43939982-0.75130219j])

In [440]:
v1*v2_new

array([-1.70599772e-04-0.02364601j, -3.94137065e-01-0.0442272j ,
       -2.39031054e-01+0.27391253j])

In [428]:
np.dot(A[:,0], B.reshape([3,1]))

array([-0.6801425-0.30155516j])

In [476]:
C = np.hstack((np.eye(3), np.array([[4],[6],[5]])))
C

array([[1., 0., 0., 4.],
       [0., 1., 0., 6.],
       [0., 0., 1., 5.]])

In [477]:
np.ufunc.reduce(C)

TypeError: descriptor 'reduce' requires a 'numpy.ufunc' object but received a 'numpy.ndarray'

In [487]:
u1 = np.array([[4],[6],[5]])
u1

array([[4],
       [6],
       [5]])

In [491]:
v = np.array([[1],[0],[0]])

u2 = np.matmul(u1,v)
u2

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 1)

In [537]:
x = np.array([4, 6, 5])
y = np.array([1, 0, 0])
w=np.cross(x,y)

w.dot(x)

0

In [540]:
np.outer(x,y)
w

array([ 0,  5, -6])

In [508]:
w

array([ 0,  5, -6])

In [502]:
z = np.cross(w,x)
z.dot(w)

0

In [506]:
AA = np.hstack((x.reshape(3,1),y.reshape(3,1),z.reshape(3,1)))
AA

array([[  4,   1,  61],
       [  6,   0, -24],
       [  5,   0, -20]])

In [529]:
def normalize(v):
    mag = sum(np.abs(v**2))
    return v / np.sqrt(mag)

def Get_arb_state_matrix(arb_state):
    
    arb_state=normalize(arb_state)
#     if sum(np.abs(arb_state**2))!=1:
#         raise ValueError('state not normalised')
        
    n_qubits=len(arb_state)
    

    
    A= np.zeros((n_qubits,n_qubits), dtype=complex)
    A[:,0]=arb_state
    
    v=np.zeros(n_qubits)
    v[0]=1
    A[:,1]=np.cross(A[:,0],v)
    
    for i in range(2, n_qubits):
        A[:,i]=np.cross(A[:,i-1], A[:,i-2])
    
    return A


In [536]:
W = Get_arb_state_matrix(np.array([4, 6, 5, 7]))

ValueError: incompatible dimensions for cross product
(dimension must be 2 or 3)

In [586]:
x1 = np.array([[4], [6], [5]])
x1=normalize(x1)
x1

array([[0.45584231],
       [0.68376346],
       [0.56980288]])

In [587]:
x2=np.array([[0], [1], [0]])

In [588]:
x2_new= x2 - np.dot(x1.T, x2)*x1

np.dot(x1.T, x2_new)

array([[2.77555756e-17]])

In [589]:
x2_new

array([[-0.31168831],
       [ 0.53246753],
       [-0.38961039]])

In [592]:
x3=np.array([[0], [0], [1]])

In [600]:
x3_new= x3 - (np.dot(x1.T, x3)*x1 + np.dot(x2_new.T, x3)*x2_new)

np.dot(x1.T, x3_new)

array([[1.66533454e-16]])

In [594]:
np.dot(x2_new.T, x3_new)

array([[-0.18215551]])

In [602]:
x2_new

array([[-0.31168831],
       [ 0.53246753],
       [-0.38961039]])

In [963]:
def Gran_Shmidt(arb_state):
    # https://stackoverflow.com/questions/12327479/how-to-build-a-ortoghonal-basis-from-a-vector
    
    arb_state=normalize(arb_state)
#     if sum(np.abs(arb_state**2))!=1:
#         raise ValueError('state not normalised')
        
    n_qubits=len(arb_state)
    A= np.eye(n_qubits, dtype=complex)
    A[:,0]=arb_state
    
    for i in range(1, n_qubits):
        project = reduce(sum, [np.inner(A[:,i], A[:,j])*A[:,j] for j in range(i)])
        
        A[:,i]=normalize(A[:,i]-project)
        
        print(np.inner(A[:,i], A[:,i-1]))
        
    return A
    

In [976]:
WW=Gran_Shmidt(np.array([1,1,0,1]))
print(WW)

np.around(WW.conj().T.dot(WW), 3)

(-1.6653345369377348e-16+0j)
0j
(-0.5298129428260175+0j)
[[ 0.57735027+0.j -0.40824829+0.j  0.        +0.j -0.52981294+0.j]
 [ 0.57735027+0.j  0.81649658+0.j  0.        +0.j -0.52981294+0.j]
 [ 0.        +0.j  0.        +0.j  1.        +0.j -0.52981294+0.j]
 [ 0.57735027+0.j -0.40824829+0.j  0.        +0.j -0.39735971+0.j]]


array([[ 1.   +0.j, -0.   +0.j,  0.   +0.j, -0.841+0.j],
       [-0.   +0.j,  1.   +0.j,  0.   +0.j, -0.054+0.j],
       [ 0.   +0.j,  0.   +0.j,  1.   +0.j, -0.53 +0.j],
       [-0.841+0.j, -0.054+0.j, -0.53 +0.j,  1.   +0.j]])

In [1001]:
from scipy.linalg import null_space
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.null_space.html
coef
Z = null_space(B)

In [1008]:
# from scipy.linalg import svd

In [1064]:
A = np.eye(3)
A[:,0]= np.array([np.sqrt(1/2), 0,np.sqrt(1/2)])
A

array([[0.70710678, 0.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.70710678, 0.        , 1.        ]])

In [1052]:
A = np.random.rand(3, 3)
project(A[:,0], A[:,1])

array([0.8423485 , 0.81449909, 0.30565563])

In [1065]:
for i in range(1, 3):
    projection = reduce(sum, [project(A[:,j], A[:,i])for j in range(i)])
        
    A[:,i]=normalize(A[:,i]-projection)

    

A


array([[ 0.70710678,  0.        , -0.70710678],
       [ 0.        ,  1.        , -0.70710678],
       [ 0.70710678,  0.        ,  0.        ]])

In [1094]:
def project(u,v):
    coef = v.conj().T.dot(u)/ u.conj().T.dot(u)
    return coef*u

def Gram_Schimdt(arb_state):
    # https://stackoverflow.com/questions/12327479/how-to-build-a-ortoghonal-basis-from-a-vector
    
#     arb_state=normalize(arb_state)
#     if sum(np.abs(arb_state**2))!=1:
#         raise ValueError('state not normalised')
        
    n_qubits=len(arb_state)
    A= np.eye(n_qubits, dtype=complex)
    A[:,0]=arb_state
    
    for i in range(1, n_qubits):
        projection = reduce(sum, [project(A[:,i], A[:,j])for j in range(i)])
        A[:,i]=A[:,i]-projection
        
    return A

In [1099]:
BB = Gram_Schimdt(np.array([np.sqrt(1/2), 0, np.sqrt(1/2)]))

np.inner(BB[:,1], BB[:,2])
BB

array([[ 0.70710678+0.j,  0.        +0.j, -0.70710678+0.j],
       [ 0.        +0.j,  1.        +0.j, -0.70710678+0.j],
       [ 0.70710678+0.j,  0.        +0.j,  0.29289322+0.j]])

In [1141]:
V = np.eye(4)
# V[:,0]= np.array([np.sqrt(1/3), np.sqrt(1/3),np.sqrt(1/3)])
V[:,0]= np.array([np.sqrt(1/2), 0,np.sqrt(1/2),-])

U=np.zeros(V.shape)
U[:,0]=V[:,0]

for i in range(1,A.shape[0]):
    U[:,i]=V[:,i]
    for j in range(i):
        U[:,i]=U[:,i]-(U[:,j].T.dot(U[:,i])/(U[:,j].T.dot(U[:,j])))*U[:,j]
        
U

array([[ 0.57735027,  0.        , -0.33333333,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.57735027,  0.        ,  0.66666667,  0.        ],
       [ 0.57735027,  0.        , -0.33333333,  0.        ]])

In [1368]:
xx=ground_state_vector.real
mag = sum(np.abs(xx**2))
new=xx / np.sqrt(mag)

In [1371]:
from scipy.linalg import svd
def Gram_Schimdt(arb_state):
    
    # Get an orthonormal basis from a single vector (defines first column of output!)
    
    #WORKING
    # https://stackoverflow.com/questions/12327479/how-to-build-a-ortoghonal-basis-from-a-vector
    
    arb_state = np.ravel(arb_state)
    
    if not np.isclose(sum(np.abs(arb_state**2)),1):
        raise ValueError('state not normalised')
        
    n_qubits=len(arb_state)
    V= np.eye(n_qubits, dtype=complex)
    V[:,0]=arb_state
    
    U=np.zeros(V.shape, dtype=complex)
    U[:,0]=V[:,0]
    for i in range(1,V.shape[0]):
        U[:,i]=V[:,i]
        for j in range(i):
            U[:,i]=U[:,i]-(U[:,j].T.dot(U[:,i])/(U[:,j].T.dot(U[:,j])))*U[:,j]
        
    Unitary_matrx, s, Vh= svd(U)
    
    # correct the sign
    if not np.allclose(Unitary_matrx[:,0], arb_state):
        Unitary_matrx[:,0]=Unitary_matrx[:,0]*-1
    
    if not np.allclose(Unitary_matrx[:,0], arb_state):
        raise ValueError('incorrect state generated')
    return Unitary_matrx


arb_input_state=new

# arb_input_state=np.array([np.sqrt(1/3), np.sqrt(1/3), np.sqrt(1/3)], dtype=complex)
N_ancilla_qubits=12
arb_input_state=np.array([random.uniform(0, 1) for _ in range(2**N_ancilla_qubits)], dtype=complex)
arb_input_state = arb_input_state/np.sqrt(arb_input_state.dot(arb_input_state)) # NORMALIZE!

Unitary_to_make_state = Gram_Schimdt(arb_input_state)

print('unitary check:', np.allclose(Unitary_to_make_state.dot(Unitary_to_make_state.T), np.eye(len(arb_input_state))))
print('correct state check:',np.allclose(Unitary_to_make_state[:,0], arb_input_state))

unitary check: True
correct state check: True


In [739]:
def Arb_state_U_gate(theta):
    Unitary_Matrix = np.array([
                    [np.cos(theta), np.sin(theta)],
                    [np.sin(theta), -1* np.cos(theta)]
                ])

    return UnitaryGate(Unitary_Matrix, label='U_gate({})'.format(np.around(theta, 3)))

In [786]:
from qiskit.circuit.library.standard_gates import XGate, YGate, ZGate
N_QUBITS=3
q_reg = QuantumRegister(N_QUBITS)
qcirc = QuantumCircuit(q_reg)

U_gate = Arb_state_U_gate(np.pi/2).control(1)
U_gate.num_ctrl_qubits=2
U_gate.num_qubits=3
U_gate.ctrl_state=2
qcirc.append(U_gate, ([0,1,2]))
qcirc.draw()

q7581_0: ────────o────────
                 │        
q7581_1: ────────■────────
         ┌───────┴───────┐
q7581_2: ┤ U_gate(1.571) ├
         └───────────────┘

In [872]:
def Get_control_parameters_IBM(num_qubits, Coefficient_list):
    if len(Coefficient_list) != 2 ** num_qubits:
        # fill missing terms with amplitude of zero!
        Coefficient_list = Coefficient_list + [0 for _ in range(2 ** num_qubits - len(Coefficient_list))]
        #raise ValueError('incorrect number of coefficients')

    state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]

    alpha_j_dict = {}
    for target_qubit in range(num_qubits - 1):

        number_controls = target_qubit

        if number_controls > 0:
            CONTROL_state_list = [Get_state_as_str(number_controls, i) for i in range(2 ** number_controls)]
        else:
            CONTROL_state_list = ['']

        term_list = []
        for control_state in CONTROL_state_list:
            top_term_str = control_state + '1'
            bottom_term_str = control_state + '0'

            top = 0
            bottom = 0
            for index, state_str in enumerate(state_list):
                if state_str[:target_qubit + 1] == top_term_str:
                    top += Coefficient_list[index] ** 2

                if state_str[:target_qubit + 1] == bottom_term_str:
                    bottom += Coefficient_list[index] ** 2
                else:
                    continue

            if (bottom == 0) and (top == 0):
                angle = 0
            else:
                try:
                    angle = np.arctan(np.sqrt(top / bottom))
                except:
                    raise ValueError('undetermined angle! NEED TO CHECK PROBLEM')

            term_list.append({'control_state': control_state, 'angle': angle})
        alpha_j_dict[target_qubit] = term_list

    ##final rotation ##
    if num_qubits!=1:
        term_list = []
        for index, state_str in enumerate([Get_state_as_str((num_qubits - 1), i) for i in range(2 ** (num_qubits - 1))]):
            control_state_str = state_str

            top_term_str = control_state_str + '1'
            bottom_term_str = control_state_str + '0'

            index_top = state_list.index(top_term_str)
            index_bottom = state_list.index(bottom_term_str)

            top = Coefficient_list[index_top]
            bottom = Coefficient_list[index_bottom]

            if (bottom == 0) and (top == 0):
                angle = 0
            else:
                try:
                    angle = np.arctan(top / bottom)
                except:
                    raise ValueError('undetermined angle! NEED TO CHECK PROBLEM')

            term_list.append({'control_state': control_state_str, 'angle': angle})

        alpha_j_dict[num_qubits - 1] = term_list

        return alpha_j_dict
    else:

        # [np.cos(self.theta), np.sin(self.theta)],         [1]  =  [a]
        # [np.sin(self.theta), -1 * np.cos(self.theta)]     [0]     [b]
        theta = np.arccos(Coefficient_list[0])
        alpha_j_dict[0] = [{'control_state': '', 'angle': theta}]

        return alpha_j_dict

In [904]:
test = Get_control_parameters_IBM(2,[np.sqrt(0.6), np.sqrt(0.1), np.sqrt(0.1), np.sqrt(0.2)])
test

{0: [{'control_state': '', 'angle': 0.5796397403637042}],
 1: [{'control_state': '0', 'angle': 0.38759668665518066},
  {'control_state': '1', 'angle': 0.9553166181245092}]}

In [920]:
relabel_dic = {index: qubitNo for index, qubitNo in enumerate(range(2)[::-1])}
type(relabel_dic[0])

int

In [949]:
def Get_state_prep_circuit_IBM2(circuit_param_dict, N_ancilla_qubits):
    
    q_reg = QuantumRegister(N_ancilla_qubits)
    qcirc = QuantumCircuit(q_reg)
    
    relabel_dic = {index: qubitNo for index, qubitNo in enumerate(range(N_ancilla_qubits)[::-1])}
    
    for qubit in circuit_param_dict:

        for term in circuit_param_dict[qubit]:
#             qubit = relabel_dic[qubit]
            if term['control_state']:
                control_value = int(term['control_state'],2)

                num_controls = len(term['control_state'])
                theta = term['angle']

                if theta == 0:
                    # yield cirq.I.on(cirq.LineQubit(qubit+self.N_system_qubits))
                    pass
                else:

                    U_gate = Arb_state_U_gate(theta).control(1)
                    U_gate.num_ctrl_qubits=num_controls
                    U_gate.num_qubits=num_controls+1
                    U_gate.ctrl_state= control_value
                    control_qubits = [relabel_dic[i] for i in range(U_gate.num_ctrl_qubits)]
                    active_qubit=relabel_dic[qubit]
                    qcirc.append(U_gate, ([*control_qubits, active_qubit]))
            else:
                theta = term['angle']
                if theta == 0:
                    continue # Identity 
                else:
                    active_qubit=relabel_dic[qubit]
                    U_gate = Arb_state_U_gate(theta)
                    qcirc.append(U_gate, ([active_qubit]))
                    
    return qcirc


In [950]:
arb_state_circuit = Get_state_prep_circuit_IBM2(test, 2)
print(arb_state_circuit.draw())

                          ┌───────────────┐┌───────────────┐
q13646_0: ────────────────┤ U_gate(0.388) ├┤ U_gate(0.955) ├
          ┌──────────────┐└───────┬───────┘└───────┬───────┘
q13646_1: ┤ U_gate(0.58) ├────────o────────────────■────────
          └──────────────┘                                  


In [951]:
unitary= Operator(arb_state_circuit)
zero_input = reduce(np.kron, [np.array([[1],[0]]) for _ in range(arb_state_circuit.num_qubits)])
unitary.data.dot(zero_input)

array([[0.77459667-5.82867088e-16j],
       [0.31622777-8.88178420e-16j],
       [0.31622777+2.17386757e-16j],
       [0.4472136 -1.13492956e-17j]])

In [936]:
import random

In [953]:
N_ancilla_qubits=3
amplitudes=np.array([random.uniform(0, 1) for _ in range(2**N_ancilla_qubits)])
amplitudes_normalized= amplitudes/np.sqrt(np.dot(amplitudes,amplitudes))

arb_control_dict = Get_control_parameters_IBM(N_ancilla_qubits,amplitudes_normalized)

arb_state_circuit = Get_state_prep_circuit_IBM2(arb_control_dict, N_ancilla_qubits)
print(arb_state_circuit.draw())

unitary= Operator(arb_state_circuit)
zero_input = reduce(np.kron, [np.array([[1],[0]]) for _ in range(arb_state_circuit.num_qubits)])
unitary.data.dot(zero_input)

                                                             ┌───────────────┐»
q13840_0: ───────────────────────────────────────────────────┤ U_gate(0.558) ├»
                           ┌───────────────┐┌───────────────┐└───────┬───────┘»
q13840_1: ─────────────────┤ U_gate(0.468) ├┤ U_gate(0.814) ├────────o────────»
          ┌───────────────┐└───────┬───────┘└───────┬───────┘        │        »
q13840_2: ┤ U_gate(0.758) ├────────o────────────────■────────────────o────────»
          └───────────────┘                                                   »
«          ┌───────────────┐┌───────────────┐┌───────────────┐
«q13840_0: ┤ U_gate(0.033) ├┤ U_gate(1.532) ├┤ U_gate(0.213) ├
«          └───────┬───────┘└───────┬───────┘└───────┬───────┘
«q13840_1: ────────o────────────────■────────────────■────────
«                  │                │                │        
«q13840_2: ────────■────────────────o────────────────■────────
«                                                             

array([[ 0.60593511-1.11022302e-15j],
       [ 0.        +0.00000000e+00j],
       [-0.40069564+1.16573418e-15j],
       [ 0.        +0.00000000e+00j],
       [-0.33583733+1.27826087e-15j],
       [ 0.        +0.00000000e+00j],
       [-0.5995823 +1.92973683e-15j],
       [ 0.        +0.00000000e+00j]])

In [947]:
sum(np.abs(amplitudes_normalized)**2)

1.0000000000000002

In [945]:
amps = [np.sqrt(0.1), 
 np.sqrt(0.2),
 np.sqrt(0.3),
 np.sqrt(0.1),
 np.sqrt(0.05),
 np.sqrt(0.05),
 np.sqrt(0.1),
 np.sqrt(0.1)]

print(amps)

arb_control_dict = Get_control_parameters_IBM(N_ancilla_qubits,amps)

arb_state_circuit = Get_state_prep_circuit_IBM(arb_control_dict, N_ancilla_qubits)
print(arb_state_circuit.draw())

unitary= Operator(arb_state_circuit)
zero_input = reduce(np.kron, [np.array([[1],[0]]) for _ in range(arb_state_circuit.num_qubits)])
unitary.dot(zero_input)

[0.31622776601683794, 0.4472135954999579, 0.5477225575051661, 0.31622776601683794, 0.22360679774997896, 0.22360679774997896, 0.31622776601683794, 0.31622776601683794]
          ┌──────────────┐┌───────────────┐┌───────────────┐┌───────────────┐»
q13356_0: ┤ U_gate(0.58) ├┤ U_gate(0.857) ├┤ U_gate(0.955) ├┤ U_gate(0.955) ├»
          └──────────────┘└───────┬───────┘└───────┬───────┘└───────┬───────┘»
q13356_1: ────────────────────────o────────────────■────────────────o────────»
                                                                    │        »
q13356_2: ──────────────────────────────────────────────────────────o────────»
                                                                             »
«          ┌───────────────┐┌───────────────┐┌───────────────┐
«q13356_0: ┤ U_gate(0.524) ├┤ U_gate(0.785) ├┤ U_gate(0.785) ├
«          └───────┬───────┘└───────┬───────┘└───────┬───────┘
«q13356_1: ────────o────────────────■────────────────■────────
«                  │        

Operator([[ 1.62635839e-01+5.65070085e-16j],
          [ 4.63148251e-02-2.49196105e-17j],
          [-9.47911137e-01+3.80482560e-15j],
          [-2.69942583e-01+1.94384899e-15j],
          [-2.72199139e-16-4.16333634e-17j],
          [-5.67468699e-17+3.47474363e-33j],
          [-4.84340011e-17+1.66533454e-16j],
          [-4.18315221e-17+2.56144198e-33j]],
         input_dims=(), output_dims=(2, 2, 2))

In [846]:
arb_control_dict

{0: [{'control_state': '', 'angle': 0.5796397403637044}],
 1: [{'control_state': '0', 'angle': 0.857071947850131},
  {'control_state': '1', 'angle': 0.9553166181245093}],
 2: [{'control_state': '00', 'angle': 0.9553166181245092},
  {'control_state': '01', 'angle': 0.5235987755982989},
  {'control_state': '10', 'angle': 0.7853981633974483},
  {'control_state': '11', 'angle': 0.7853981633974483}]}